In [11]:
# LLM 모델을 통한 연산 예제
import os
import requests
from typing import Type
from pydantic import BaseModel,Field
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0)

alpha_vantage_api_key = os.environ.get("AlPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for") # Field는 각 필드의 메타데이터(추가 설명 등)를 제공

# 주식 심볼 찾는 도구(종목 코드)
class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for Apple Company
    """

    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        # DuckDuckGo 검색 엔진을 통해 웹 검색을 수행하는 기능 또는 도구
        search_engine = DuckDuckGoSearchAPIWrapper()
        # 검색 엔진을 사용하여 웹 검색
        return search_engine.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock symbol of the company. Example: AAPL, TSLA",)


# 회사 개요 찾는 도구
class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        api_request_result = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return api_request_result.json()

agent = initialize_agent(
    llm=llm,
    verbose = True, 
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        # 종목 코드 검색 도구
        StockMarketSymbolSearchTool(),
        # 회사 개요 탐색 도구
        CompanyOverviewTool(),
    ]
)

prompt = "Give me financial information on Cloudflare's stock and help me analyze if it's a potential good investment. Answer must be in Korean."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare'}`


Read the latest posts from Cloudflare's official blog, covering topics such as security, performance, AI, email, DNS, and more. Learn how Cloudflare disrupts threats, innovates features, and improves Internet infrastructure. We have, over the last 24 hours, released an automatic JavaScript URL rewriting service that will rewrite any link to polyfill.io found in a website proxied by Cloudflare to a link to our mirror under cdnjs. This will avoid breaking site functionality while mitigating the risk of a supply chain attack. Following its initial announcement in September 2022, Cloudflare Calls is now in open beta and available in your Cloudflare Dashboard.Cloudflare Calls lets developers build real-time audio/video apps using WebRTC, and it abstracts away the complexity by turning the Cloudflare network into a singular SFU.In this post, we dig into how we make this possible.

{'input': "Give me financial information on Cloudflare's stock and help me analyze if it's a potential good investment. Answer must be in Korean.",
 'output': '클라우드플레어(Cloudflare)는 기술 기업으로, 미국 캘리포니아에 본사를 두고 있습니다. 주식 코드는 NET이며 뉴욕 증권거래소에 상장되어 있습니다. 최근 연간 매출은 13억 8517만 2000달러이며, 시가총액은 287억 2270만 달러입니다. 주당 순이익은 -0.53달러이며, 주당 순자산은 2.347달러입니다. 클라우드플레어의 주가는 52주 최고가가 116달러이고, 52주 최저가가 53.88달러입니다. 주식의 50일 이동평균은 77.2달러이며, 200일 이동평균은 79.32달러입니다. 분석가들은 클라우드플레어의 주식에 대해 강력매수 5개, 매수 9개, 보유 14개, 판매 3개, 강력판매 1개의 평가를 내리고 있습니다. 미래 주가 수익비율은 158.73이며, 주식의 가치 평가는 90.93달러입니다. 클라우드플레어의 주식은 현재 안정적인 투자로 평가될 수 있습니다.'}